> Required Imports

In [1]:
import pandas as pd
import librosa
import librosa.display
from os.path import getsize
from os import remove
from IPython.display import Audio
from config import CONFIG

from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.models import Sequential
import numpy as np
from gtts import gTTS
from tensorflow import keras

import warnings
warnings.filterwarnings('ignore')

> Read the voice data and manually pick out ~3 audios for each celebrity where the speech can easily be transcribed

In [ ]:
df = pd.read_csv(CONFIG.get("files", "base_data"))

In [4]:
convert_to_signature = df.iloc[
    [
        0,  1,  2,
        205,207,208,
        388,390,391,
        519,524,525,
        692,694,698,
        844,845,849,
        1048,1044,1051,
        1242,1243,1248,
        1367,1368,1369,
        1567,1575,1576,
        1656,1659,1660,
        1856,1857,1858,
        1972,1974,1976,
        2179,2183,2126,
        2277,2278,2281,
        2477,2478,2479,
        2665,2666,2667,
        2865,2866,2867,
        3065,3071,3067,
        3266,3267,3271,
        3386,3387,3393,
        3558,3563,3565,
        3760,3761,3762,
        3958,3959,3967,
        4164,4165,4167,
        4215,4216,4217
    ]
]

In [ ]:
convert_to_signature["text"] = ""

In [6]:
convert_to_signature.head()

,speaker_id,filepath,text
0,id10000,..\data\vox1_indian\id10000\GoogleSamples\0000...,
1,id10000,..\data\vox1_indian\id10000\GoogleSamples\0009...,
2,id10000,..\data\vox1_indian\id10000\GoogleSamples\0019...,
205,id10001,..\data\vox1_indian\id10001\ProfAudio\audio13.wav,
207,id10001,..\data\vox1_indian\id10001\ProfAudio\audio45.wav,


> Find the duration to which all audio files will be cut

In [7]:
smallest_file = min(df["filepath"], key=getsize)
y, sr = librosa.load(smallest_file)
y, _ = librosa.effects.trim(y)
duration = librosa.get_duration(y=y, sr=sr)
display(duration)

3.960090702947846

> Manually transcribe the audio to match google assistant as closely as possible (Even if it means, including a misspelled transcription)

In [8]:
convert_to_signature = convert_to_signature.copy()
texts = [
    "substantial cartridge offense proliferation fancy maternity",
    "afraid galoshes wets vegetarian breezy zinc interlace",
    "nebulous malar trips here curly collateral raises chad",
    "you know maybe it's in a more severe stage, uh, as and as supposed",
    "think about... the columns are the features, right?",
    "each direction of the output! then you need to break it up into multiple",
    "came back to me and then again went back to hire us, so it should not be like",
    "e you know people will be overcritical about it I think people kinda understood",
    "know that whatever I'm doing is being loved by people and I do get",
    "now what I mean is the the the script we had in front of us all of us spent",
    "um, I don't do too much to look after myself, but what I do",
    "yes irrespective of what religion they come from and a few",
    "understand and get out of it, lot of people like like I'm seeing now the whole industry",
    "don't care what I'm doing, when you really don't care what I've had to",
    "buh imin general people! I really believe that people talk rubbish",
    "uh he's not uh uh a guy who gets happy very fast; some uh",
    "with a vapas you you become normal and you start playing that character much faster",
    "uhh gets it from outside and she writes it down, and a",
    "mujhe kaam mil raha hai or kuch log merei saath kaam karna chaate hai",
    "countries like united states of america russia meh, china such",
    "I just consider myself very fortunate to have uh, being able to uh",
    "gap between the disney one and uh, our warner brothers one, umm I",
    "I'd just broken up with my boyfriend it was really, traumatic is pressed up",
    "uh they fall in love because of the similar interest where she reelly",
    "body starts giving signal, those kind of singals you know uh, where",
    "that you know aap ko lagta hai key yaar ka e baar bolt eh key aapko moks mil jaaye",
    "daniel lewis uh, Phillips similar hopper which is not there, mcbrando which",
    "I know, that I have things to say that will be very important",
    "kals so I think we need more of this lot I am designing scripts",
    "and and we've shot every action sequence and then we've shot it here that I believe",
    "action meh toh mujhe itna lagta nai hai key interest hai lekin direction",
    "league and people really look up to, and the the the way advertisements",
    "and publicly, individual who doesn't have an iota of respect",
    "achieveable it's doable because uh, hollywood also a post slum",
    "on a, the late indira gandhi, I would like to do",
    "all wrong which none of that is moralistic she's using her body",
    "or to serve art in it, we all become actors because we are",
    "xx",
    "to people with no resources, and where",
    "so after about 6 months, in fact I was in",
    "so english used to bother me and there it was a big",
    "mujhe mathematics may physics may or in sub subjects may marks bot",
    "obviously uh, uh, with a title a poster you invite",
    "uh but whatever heroin to say so, I really want to work with him",
    "don't ever want to watch a film and look look for myself in it, y'know I don't want to just",
    "you know he would make you do all kinds of things and and really an",
    "happily. yeah! and not to get to umm",
    "a and then of course asking kamal sir y'know what he thought about the character what he wanted",
    "gin key unhe coy b pressure moment may daaldo",
    "okay I was comfortable but then I realized with men especially in sports",
    "a and not being derogatory to them, that's not my friends also",
    "missions friendships so think ye javani hay divani embodies the entire film",
    "a he has a little bit popularity on the internet he becomes like an internet sensation",
    "eh because I'm playing a hero for the first time, you know a larger than life carrot if you see all my",
    "I mean that's also a great thing and real stardom comes beyond the roads",
    "so it is not just me as an actor with Erin there's a whole team",
    "should be, it should be a problem because what?",
    "it's about how well two people can understand each other and how much they really wanna",
    "I'm doing my own thing I have my own career I have my own life and nobody",
    "xx",
    "instantly in love with it eck romantic thriller it ney time k baad",
    ". I'm known as uh, at home my my mum my",
    "also, so again I think that that is something that is..",
    "other product is gonna be, a close to me or I can emulate that",
    "background where I'm really trained as an actor, or uh uh been on sets",
    "uhh, so yeah man I think I would like to invite some people next time to come in and try",
    "because even if my father might not agree with my mom privately but when they act",
    "black, I used to like beg the assistants to like take me and pick me up and all it's",
    "thing is umm, I don't know he never reacts to this but I've",
    "think that's something that's been played with even though, it's not to the",
    "and you get an idea of how hard they must've what were we trying to do we were trying to",
    "thing because you know it's not it's not a book that gives you a happy feeling or a book that gives you a sad",
    "yeah, I was it was too exciting for me to get to do this role and to",
    "the interesting thing about, the way this character",
    "very important backstory that he's added to begum's",
    "when I looked at myself in the mirror I couldn't believe it was me",
    "this phase has taught me is that you have to just keep going on, so summary",
    "kush hota hay in ka I think everyone has, that! y'know",
]

In [9]:
convert_to_signature["text"] = texts
convert_to_signature[convert_to_signature["text"] == "xx"]

In [12]:
# Manual Data Correction for transcription
convert_to_signature = convert_to_signature.drop(1974)
convert_to_signature.loc[3271, "filepath"] = convert_to_signature.loc[3266, "filepath"].replace("00003", "00004")
convert_to_signature.loc[3271, "text"] = "I mean that's what I've learnt from life like everything's a phase everything that's true today will not be true tomorrow"
convert_to_signature.loc[len(convert_to_signature)] = convert_to_signature.loc[2183]
convert_to_signature.loc[77, "text"] = "and I, got major row-there was one problem which I was"
convert_to_signature["speaker_id"].value_counts()

> Convolute the audio and Google Assistant's version of the same audio and get the difference as the signature

In [20]:
# Load trained recognition model
loaded_model = keras.models.load_model(CONFIG.get("files", "recog_model"))

# Get layerwise weights
weights = []
for layr in loaded_model.layers:
    weights.append(layr.get_weights())

# Build only the convolutional part of the network and assign the trained weights
model = Sequential()
model.add(Conv2D(input_shape=(128,171,1),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))

for i, layr in enumerate(model.layers):
    model.layers[i].set_weights(weights[i])
    model.layers[i].trainable = False

> Get the embeddings of the speeches WRT Google's speech

In [25]:
def conv(ps):
    # Convolute the Spectrogram to get a convoluted 4096 dimensional output (input to decoder)
    return model.predict(np.array([ps.reshape((128,171,1))]), verbose=0)[0]

In [138]:
language = 'en'
google_filepath = CONFIG.get("files", "reconst_models") # Any directory where we can create and delete the "temporary" google audio file

def get_embedding(row):
    # User Spectrogram
    y, sr = librosa.load(row["filepath"])
    y = y[:int(sr*duration)]
    temp = len(y)
    ps = librosa.feature.melspectrogram(y=y, sr=sr)
    
    # Google Spectrogram
    tts = gTTS(text=row["text"], lang=language, slow=False)
    tts.save(google_filepath + "temp.wav")
    y, sr = librosa.load(google_filepath + "temp.wav")
    remove(google_filepath + "temp.wav")
    speed_factor = (len(y)/int(sr*duration))**(0.25)
    y_new = librosa.effects.time_stretch(y, speed_factor)
    if len(y_new) > int(sr*duration):
        y_new = y_new[:int(sr*duration)]
    elif len(y) > int(sr*duration):
        y_new = y[:int(sr*duration)]
    else:
        y_new = librosa.util.fix_length(y, size=int(sr*duration))
    ps_google = librosa.feature.melspectrogram(y=y_new, sr=sr)

    if temp != len(y_new):
        display(row)

    return conv(ps) - conv(ps_google)
    
embedding = get_embedding(convert_to_signature.iloc[50])
embedding

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [158]:
embeddings = convert_to_signature.apply(lambda x: get_embedding(x), axis=1)
embeddings_df = pd.DataFrame(dict(zip(embeddings.index, embeddings.values))).T
embeddings_df = pd.concat([embeddings_df, convert_to_signature], axis=1).drop(columns=["filepath", "text"])
embeddings_df = embeddings_df.groupby("speaker_id")[[*range(4096)]].mean().reset_index()
embeddings_df.to_csv(CONFIG.get("files", "speaker_embedding"), index=False)

In [177]:
embeddings_df.head()

,speaker_id,0,1,2,3,4,5,6,7,8,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,id10000,0.0,0.0,0.0,0.077678,0.0,0.0,0.000000,-0.167808,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.196701,0.0,0.0,0.000000
1,id10001,0.0,0.0,0.0,0.795374,0.0,0.0,0.512082,-0.002412,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.693631,0.0,0.0,0.000000
2,id10002,0.0,0.0,0.0,-0.576427,0.0,0.0,0.391755,-0.077004,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.342671,0.0,0.0,0.000000
3,id10003,0.0,0.0,0.0,0.246919,0.0,0.0,0.293667,-0.105879,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.275015,0.0,0.0,0.000000
4,id10017,0.0,0.0,0.0,-0.458128,0.0,0.0,0.922944,0.021755,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.131275,0.0,0.0,0.248438
